In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> GERAR DADOS

# import importlib
# importlib.reload(queries)

from src.db_connection import get_connection
from queries import get_query_discagens, get_query_cad_devf, get_query_mailing_hist, get_query_discagens_trestto
import pandas as pd

conn_src = get_connection("SERVER_SRC", "DATABASE_SRC")
# Tudo em uma linha
df_discagens_expert = pd.read_sql(
    get_query_discagens('2025-09-01', '2025-09-30'), 
    conn_src
)
df_cad_devf = pd.read_sql(
    get_query_cad_devf(), 
    conn_src
)

conn_bd2 = get_connection("SERVER_BD2", "DATABASE_BD2")
df_maling_hist = pd.read_sql(
    get_query_mailing_hist('2025-09-01', '2025-09-30'), 
    conn_bd2
)

conn_trc = get_connection("SERVER_BD2", "DATABASE_TRC")
df_discagens_trestto = pd.read_sql(
    get_query_discagens_trestto('2025-09-01', '2025-09-30'), 
    conn_trc
)

C:\Users\claudiano.alves\AppData\Local\Temp\ipykernel_26960\4139424898.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_discagens = pd.read_sql(
C:\Users\claudiano.alves\AppData\Local\Temp\ipykernel_26960\4139424898.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cad_devf = pd.read_sql(
C:\Users\claudiano.alves\AppData\Local\Temp\ipykernel_26960\4139424898.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_maling_hist = pd.read_sql(
C:\Users\claudiano.alves\AppData\Local\Temp\ipykernel_26960\4139424898.p

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> TRATAMENTO DOS DADOS
from src.data_wrangling_discagens_expert import tratar_base_discagens
from src.data_wrangling_mailingHist import tratar_base_mailing_hist
from src.data_wrangling_discagens_trestto import tratar_discagens_trestto

# Aplicar tratamento
# df_discagens_expert = tratar_base_discagens(df_discagens)
# df_maling_hist = tratar_base_mailing_hist(df_maling_hist)
# df_trestto_esforco, df_trestto_unique = tratar_discagens_trestto(df_discagens_trestto)

df_trestto_esforco, df_trestto_unique = tratar_discagens_trestto(
    df_discagens_trestto, 
    df_maling_hist
)

# df_maling_hist.head()
#df_cad_devf.head()

# df_mailing_hist_cruz = tratar_base_discagens(df_mailing_hist)

#df_discagens.head()

print(f"✓ Esforço diário: {len(df_trestto_esforco)} dias")
print(df_trestto_esforco.head())

print(f"\n✓ Únicos diário: {len(df_trestto_unique)} dias")
print(df_trestto_unique.head())

📊 Antes da consolidação - Trestto: 2,762,568
📊 Após consolidação - Trestto: 2,648,475
📊 Mailing: 6,255,364
📊 Após join: 1,099,803


c:\Users\claudiano.alves\Documents\Claudiano\repository\Data-Analysis-Portfolio\Projects\Ouze\src\data_wrangling_discagens_trestto.py:39: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(f"📊 Após join: {len(df_join):,}")
c:\Users\claudiano.alves\Documents\Claudiano\repository\Data-Analysis-Portfolio\Projects\Ouze\src\data_wrangling_discagens_trestto.py:49: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  


✓ Esforço diário: 750 dias
         DATA PRODUTO FX_ATRASO  DISCAGEM   ALO  CPC  CPCA  PROMESSA
0  2025-09-01     API   Menor 0         0     0    0     0         0
1  2025-09-01     API      0-30    105370  3337  526   322       186
2  2025-09-01     API     31-60     24001   461   42    27        12
3  2025-09-01     API     61-90         0     0    0     0         0
4  2025-09-01     API    91-120     19075   343   33    17        12

✓ Únicos diário: 750 dias
         DATA PRODUTO FX_ATRASO  DISCAGEM   ALO  CPC  CPCA  PROMESSA
0  2025-09-01     API   Menor 0         0     0    0     0         0
1  2025-09-01     API      0-30     29489  2645  505   312       186
2  2025-09-01     API     31-60      7385   376   42    27        12
3  2025-09-01     API     61-90         0     0    0     0         0
4  2025-09-01     API    91-120      7147   299   33    17        12


In [35]:
# duplicados_por_data = df_discagens_trestto.duplicated(subset=['CPF', 'DATA'], keep=False)
# total_duplicados = duplicados_por_data.sum()

# print(f"Total de CPFs duplicados por data: {total_duplicados}")
df_trestto_esforco_menor_0 = df_trestto_esforco.groupby(['DATA', 'FX_ATRASO'])['DISCAGEM'].sum().reset_index()
df_trestto_esforco_menor_0.head()


# Seleciona as colunas desejadas
# colunas_exibir = ['DATA', 'CPF', 'DISCAGEM', 'ALO', 'CPC', 'CPCA', 'PROMESSA']

# # Filtra os registros duplicados por CPF e DATA
# duplicados = df_discagens_trestto[df_discagens_trestto.duplicated(subset=['CPF', 'DATA'], keep=False)]

# # Cria o DataFrame com as colunas desejadas
# df_duplicados = duplicados[colunas_exibir].sort_values(by=['DATA', 'CPF'])

# # Exibe os dados duplicados
# print(df_duplicados)


C:\Users\claudiano.alves\AppData\Local\Temp\ipykernel_26960\2858724473.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_trestto_esforco_menor_0 = df_trestto_esforco.groupby(['DATA', 'FX_ATRASO'])['DISCAGEM'].sum().reset_index()


,DATA,FX_ATRASO,DISCAGEM
0,2025-09-01,Menor 0,0
1,2025-09-01,0-30,105370
2,2025-09-01,31-60,24001
3,2025-09-01,61-90,0
4,2025-09-01,91-120,19075


In [50]:
from src.data_wrangling_mailingHist import tratar_base_mailing_hist, adicionar_valor_principal
#df_cad_devf.head()
#df_cad_devf_ativo = df_cad_devf[df_cad_devf['STATCONT_FIN'] == 0]
df_mailing_hist_valor = adicionar_valor_principal(df_maling_hist, df_cad_devf)
#df_cad_devf_ativo.head()
df_mailing_hist_valor.head()

📊 Antes do join - Mailing: 6,255,364 | CAD_DEVF: 2,996,445
📊 Após join: 6,255,364
📊 Contratos com valor: 6,248,734
📊 Contratos sem valor: 6,630


,DATA,CONTRATO,CPF,ATRASO,COD_CLI,COD_CAR,PRODUTO,FX_ATRASO,VALORPRIN_FIN
0,2025-09-01,41976615ID,11197448730,452,228,2,API,361-720,461.79
1,2025-09-01,37087148ID,372010000158,205,228,2,API,181-360,59.80
2,2025-09-01,48331916ID,717568000128,24,228,2,API,0-30,29.90
3,2025-09-01,30033123ID,1853892000136,200,228,2,API,181-360,443.80
4,2025-09-01,52227058ID,1467973000106,53,228,2,API,31-60,79.90


In [51]:
df_sem_valorprin = df_mailing_hist_valor[df_mailing_hist_valor['VALORPRIN_FIN'].isna()]
df_sem_valorprin.head()

,DATA,CONTRATO,CPF,ATRASO,COD_CLI,COD_CAR,PRODUTO,FX_ATRASO,VALORPRIN_FIN
2867,2025-09-01,48745367ID,30009434000113,216,198,1,Agenda Negativa,181-360,NaN
3336,2025-09-01,48850260ID,31878313000170,209,198,1,Agenda Negativa,181-360,NaN
3942,2025-09-01,47404680ID,33489743000126,251,198,1,Agenda Negativa,181-360,NaN
4139,2025-09-01,48278436ID,33941886000127,238,198,1,Agenda Negativa,181-360,NaN
4930,2025-09-01,47803916ID,36444760000135,245,198,1,Agenda Negativa,181-360,NaN


In [ ]:


#df_discagens.head()

,DATA,id,CONTRATO,CPF,ddd,fone,GrupoPrincipal,UltCodSigRecPublica,ResultadoClassificacao,MotivoEncerramentoBilhete,Instante200OKPub,Agente,tempoconversacao_ms,OPERACAO,ESTADO
0,2025-09-01,66,48941489id,43640864000180,27,998267780,4047,480,,foneforadeservico,1899-12-30 00:00:00,,0,URA CPC,ES
1,2025-09-01,149,49012660id,57722707000129,11,945760003,4636,480,,foneforadeservico,1899-12-30 00:00:00,,0,AGV NEGOCIADORA,SP
2,2025-09-01,196,47525229id,55280229000119,12,996448349,4636,404,,foneinexistente,1899-12-30 00:00:00,,0,AGV NEGOCIADORA,SP
3,2025-09-01,203,42643945id,54542547000148,45,999946130,4636,404,,foneinexistente,1899-12-30 00:00:00,,0,AGV NEGOCIADORA,PR
4,2025-09-01,220,41849079id,53330359000193,62,999623988,4047,200,cxpre,bloqueiodetector,2025-09-01 08:00:09,,0,URA CPC,GO


In [27]:
# # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> CARREGANDO BASE DE ACIONAMENTOS <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#
df_maling_hist.head()
# print(df_discagens_trestto.nunique())
# print(df_discagens_trestto.groupby('DATA')['CPF'].nunique().reset_index(name='CPFs_unicos')
# )
# print(len(df_discagens_trestto))
# print(len(df_maling_hist))
#df_maling_hist.head()
# query = f"""
#     SELECT 
# 		CAST(A.DATA_ACIONA AS DATE) DATA_ACIONA,
#         CAST(DATA_ACIONA2 AS DATE) AS DATA,
#         A.CONTRATO_FIN,
#         C.CPF_DEV,
#         B.COD_ACIONAMENTO,
#         B.DESC_ACIONAMENTO,
#         C.DTDEVOL_FIN,
#         C.VALORPRIN_FIN,
#         C.VALOR_FIN,
#         C.COD_CLI,
#         C.COD_CAR,
#         'HUMANO' TIPO,
#     FROM ACIONA D
#     LEFT JOIN CAD_ACIONAMENTO B ON A.COD_ACIONAMENTO = B.COD_ACIONAMENTO
#     WHERE CAST(DATA_ACIONA2 AS DAT4) BETWEEN '{dt_ini}' AND '{dt_fim}' AND COD_RECUP NOT IN (15721)
# """
# df_aciona = pd.read_sql(query, conn)
# df_aciona.head()

# df_aciona = pd.read_sql(f"SELECT DATA_ACIONA2 FROM ACIONA", conn)
# df_aciona.head()

,DATA,CONTRATO,CPF,ATRASO,COD_CLI,COD_CAR,PRODUTO,FX_ATRASO
0,2025-09-01,41976615ID,11197448730,452,228,2,API,361-720
1,2025-09-01,37087148ID,372010000158,205,228,2,API,181-360
2,2025-09-01,48331916ID,717568000128,24,228,2,API,0-30
3,2025-09-01,30033123ID,1853892000136,200,228,2,API,181-360
4,2025-09-01,52227058ID,1467973000106,53,228,2,API,31-60


In [47]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>> TESTES <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# tipo da coluna
from src import data_wrangling_discagens_trestto
from src import data_wrangling_mailingHist
import importlib
importlib.reload(data_wrangling_discagens_trestto)
importlib.reload(data_wrangling_mailingHist)
# >>>>>>>>>>>>>>>>>>>>>>> VERIFICAR TIPO DE DADO DE UMA COLUNA ESPECÍFICA
#print(df_discagens['ddd'].dtype)
# primeiros valores (mostra índices também)
#df_discagens['ddd'].head(30)

# >>>>>>>>>>>>>>>>>>>>>>> MOSTRAR PARTE DO DATAFRAME
# df_discagens.head()

# >>>>>>>>>>>>>>>>>>>>>>> FILTRAR DADOS DE UMA COLUNA ESPECÍFICA
# df_ativo = df_mailing_hist[df_mailing_hist['ID_CAR'].notna()]
# df_ativo.head()

# import pandas as pd
# Supondo que seu DataFrame seja 'df'

# valores_unicos = df_cad_devf['DESC_CAR'].unique()
# print(valores_unicos)

# >>>>>>>>>>>>>>>>>>>>>>> CONTAR VALORES ÚNICOS DE UMA COLUNA ESPECÍFICA
# len(df_ativo)
# df_discagens['OPERACAO'].value_counts().reset_index().rename(columns={'index': 'OPERACAO', 'OPERACAO': 'QTD'})
#  print(filtro)
#len(df_discagens)
#print("Linhas:", df_discagens.shape[0], " | Colunas:", df_discagens.shape[1])
#df_discagens.columns.tolist()
#df_discagens['Agente'].unique()
#type(df_discagens)

# >>>>>>>>>>>>>>>>>>>>>>> resumo por estado
# df_resumo = (
#     df_discagens
#     .groupby('ESTADO')['CPF']
#     .nunique()
#     .reset_index()
#     .rename(columns={'CPF': 'QTD_CPF_DISTINTOS'})
#     .sort_values(by='QTD_CPF_DISTINTOS', ascending=False)
# )
# df_resumo.head()
# print(df_resumo)
# >>>>>>>>>>>>>>>>>>>>>>>>>>> TESTANDO AS CONEXÕES <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# server = os.getenv('SERVER_BD2')
# database = os.getenv('DATABASE_TRC')
# print(f"Valor do Servidor: {server}")
# print(f"Valor do Banco de Dados: {database}") 

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> RECARREGANDO O .ENV <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#
# from dotenv import load_dotenv
# import os

# # Recarrega o .env
# load_dotenv(dotenv_path='.env', override=True)

# # Testa se as variáveis foram carregadas
# print(os.getenv("SERVER_BD2"))
# print(os.getenv("DATABASE_TRC"))

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> AGRUPANDO DADOS COM SOMA (TRESTTO) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#
#df_discagens_trestto = df_discagens_trestto.groupby('DATA')['ALO'].sum().reset_index()


# data_inicial = '2025-09-30'
# data_final = '2025-09-30'

# Filtrar entre datas
# df_discagens_trestto_sum = df_discagens_trestto_sum[(df_discagens_trestto_sum['DATA'] >= data_inicial) & (df_discagens_trestto_sum['DATA'] <= data_final)]

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> FECHANDO AS CONEXÕES <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#
# if 'conn' in locals() and conn:
#     conn.close()
#     print("Conexão com o banco de dados fechada com sucesso.")


<module 'src.data_wrangling_mailingHist' from 'c:\\Users\\claudiano.alves\\Documents\\Claudiano\\repository\\Data-Analysis-Portfolio\\Projects\\Ouze\\src\\data_wrangling_mailingHist.py'>